In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# charts
import seaborn as sns 
import matplotlib.pyplot as plt
import squarify #TreeMap

# import graph objects as "go"
import plotly.graph_objs as go

# For exercise 4 and 5
import plotly.express as px

#for offline plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#for in-notebooks plot using plotly
init_notebook_mode(connected=True)

%matplotlib inline
from IPython.display import display

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import chart_studio.plotly as py

In [9]:
#!pip install chart-studio==1.0.0
import chart_studio
chart_studio.tools.set_credentials_file(username='23091998', api_key='6HCxpAH5r6mBlYoC2wIM')

In [10]:
Test_1 = pd.read_csv("Data/transfer_data_Final_Master_2010_2011_Jul.csv")
Test_2 = pd.read_csv("Data/transfer_data_Final_Master_2011_Aug_2012_Jun.csv")
Test_3 = pd.read_csv("Data/transfer_data_Final_Master_2012_Jul_2015.csv")
Test_4 = pd.read_csv("Data/transfer_data_Final_Master_2016_Aug_Dec.csv")
Test_5 = pd.read_csv("Data/transfer_data_Final_Master_2016_Jan_Jul.csv")
Test_6 = pd.read_csv("Data/transfer_data_Final_Master_2017.csv")
Test_7 = pd.read_csv("Data/transfer_data_Final_Master_2018.csv")
Test_8 = pd.read_csv("Data/transfer_data_Final_Master_2019.csv")
Test_9 = pd.read_csv("Data/transfer_data_Final_Master_2005.csv")
Test_10 = pd.read_csv("Data/transfer_data_Final_Master_2006.csv")
Test_11 = pd.read_csv("Data/transfer_data_Final_Master_2008.csv")
Test_12 = pd.read_csv("Data/transfer_data_Final_Master_2009.csv")
Test_13 = pd.read_csv("Data/transfer_data_Final_Master_2007.csv")
Test_14 = pd.read_csv("Data/transfer_data_Final_Master_2007_Apr.csv")
Test_EDA = pd.concat([Test_1, Test_2, Test_3,
                      Test_4, Test_5, Test_6, 
                      Test_7, Test_8, Test_9, 
                      Test_10, Test_11, Test_12, 
                      Test_13, Test_14], ignore_index=True)

In [11]:
# Cleaning the name for the column Club left
New = Test_EDA['Club Left'].str.replace('\n', '')
New = New.to_frame(name = 'Club_Left')

# Cleaning the name for the column Club Joined
New1 = Test_EDA['Club Joined'].str.replace('\n', '')
New1 = New1.to_frame(name = 'Club_Joined')

# Cleanning Transfer Data
New2 = Test_EDA['Transfer'].str.replace('?', '0')
New2 = New2.to_frame(name = 'Transfer_Fee')

New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Free transfer","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("-","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(",","")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" mil. €","0000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" K €","000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" €","")

# Cleaning Market Valuation data
New3 = Test_EDA['Market Valuation'].str.replace(',', '')
New3 = New3.to_frame(name = 'Market_Valuation')
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" mil. €","0000")
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" K €","000")

#Replacing with original data
Test_EDA["Club Left"]=New["Club_Left"]
Test_EDA["Club Joined"]=New1["Club_Joined"]
Test_EDA["Transfer"]=New2["Transfer_Fee"]
Test_EDA["Market Valuation"]=New3["Market_Valuation"]

# Converting the Date from object ot a datetime format
Test_EDA['Date'] = pd.to_datetime(Test_EDA['Date'], errors='coerce')

# Removing retired players
Test_EDA.rename(columns={"Club Joined": "Club_Joined"}, inplace=True)
Test_EDA=Test_EDA[~Test_EDA.Club_Joined.str.contains("Retired")]
Test_EDA.rename(columns={"Club_Joined": "Club Joined"}, inplace=True)

Test_EDA.reset_index(inplace=True)

#Converting to float
Test_EDA["Transfer"]=Test_EDA['Transfer'].apply(lambda x:float(x))
Test_EDA["Market Valuation"]=Test_EDA["Market Valuation"].apply(lambda x:float(x))
Test_EDA["Age"]=Test_EDA['Age'].apply(lambda x:float(x))

#Adding surplus/deficit column
Test_EDA.rename(columns={"Market Valuation": "Market_Valuation"}, inplace=True)
Test_EDA["Surplus/Deficit"]=Test_EDA.Transfer-Test_EDA.Market_Valuation
Test_EDA.rename(columns={"Market_Valuation": "Market Valuation"}, inplace=True)

#Adding year column
Test_EDA["Year"]=pd.DatetimeIndex(Test_EDA["Date"]).year

Test_EDA

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,Market Valuation,Transfer,Surplus/Deficit,Year
0,0,2010-01-01,Douglas Costa,19.0,Right Winger,Brazil,Grêmio,Série A,Shakhtar D.,NaN,4000000.0,8000000.0,4000000.0,2010
1,1,2010-01-01,Florent Sinama-Pongolle,25.0,Second Striker,France,Atlético Madrid,LaLiga,Sporting CP,NaN,7000000.0,6500000.0,-500000.0,2010
2,2,2010-01-01,Alex Teixeira,19.0,Left Winger,Brazil,Vasco da Gama,Série A,Shakhtar D.,NaN,3500000.0,6000000.0,2500000.0,2010
3,3,2010-01-01,Keisuke Honda,23.0,Attacking Midfield,Japan,VVV-Venlo,Eredivisie,CSKA Moscow,NaN,3000000.0,6000000.0,3000000.0,2010
4,4,2010-01-01,Younès Kaboul,23.0,Centre-Back,France,Portsmouth,Premier League,Spurs,NaN,5700000.0,5900000.0,200000.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61835,63255,2007-09-25,Thomas Sowunmi,29.0,Centre-Forward,Hungary,Without Club,NaN,Vasas FC,NaN,300000.0,0.0,-300000.0,2007
61836,63256,2007-09-28,Onur Karali,25.0,Midfielder,Turkey,Osmaniyespor,Turkey\t\t,Altinova Bld,NaN,200000.0,0.0,-200000.0,2007
61837,63257,2007-09-28,Enver Isik,22.0,Right Winger,Turkey,Kayserispor,Süper Lig,Göztepe,NaN,250000.0,0.0,-250000.0,2007
61838,63258,2007-09-28,Mustafa Özzengi,20.0,Goalkeeper,Turkey,Galatasaray U21,Turkey\t\t,Bakirköyspor,NaN,50000.0,0.0,-50000.0,2007


In [12]:
# Top 20 clubs by spending
df=Test_EDA.groupby("Club Joined").agg({"Transfer":"sum"})
df = df.sort_values(by='Transfer', ascending = False)
df = df[:15]
df

,Transfer
Club Joined,
Man City,1.905340e+09
Real Madrid,1.697900e+09
FC Barcelona,1.630070e+09
Chelsea,1.573880e+09
Man Utd,1.407130e+09
Paris SG,1.297100e+09
Juventus,1.281790e+09
Liverpool,1.222448e+09
Atlético Madrid,1.114510e+09


In [13]:
merged_df_x = pd.merge(df.reset_index(),Test_EDA.reset_index(),
                    how='inner',
                    on = 'Club Joined')
merged_df_x_2 = merged_df_x.groupby(['Club Joined', 'Year']).agg({'Transfer_y':'sum'})
merged_df_x_2.rename(columns={"Transfer_y": "Total Fee Spent"}, inplace=True)
merged_df_x_2

Total Fee Spent
Club Joined Year                 
AC Milan    2005       33500000.0
            2006       15000000.0
            2007       47600000.0
            2008       43400000.0
            2009       25000000.0
...                           ...
Spurs       2015       66730000.0
            2016       82400000.0
            2017       80100000.0
            2018       28400000.0
            2019       98000000.0

[221 rows x 1 columns]

In [14]:
fig4a = px.line(merged_df_x_2.reset_index(), 
                x = "Year", 
                y = "Total Fee Spent",
                color = "Club Joined",
                color_discrete_sequence = px.colors.cyclical.IceFire,
                hover_name = "Club Joined", 
                title = 'Time-series Trend for Top 15 Aggregate Spending Clubs',
                template = 'plotly_dark')

fig4a.update_layout(
    hoverlabel=dict(
        bgcolor="black",
        font=dict(
                family="Arial, monospace",
                size=14,
                color="white"
            ),
        bordercolor = "black"
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Arial, monospace",
                size=16,
                color="white"
            )
        ),
        showline = True,
        linecolor = 'white',
        showticklabels = True,
        tickfont = dict(
            family = 'Arial',
            size = 14,
            color = 'white'
      )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Transfer Fee Spent",
            font=dict(
                family="Arial, monospace",
                size=16,
                color="white"
            )
        ),
        showline = False,
        linecolor = 'white',
        showticklabels = True,
        tickfont = dict(
            family = 'Arial',
            size = 14,
            color = 'white'
        )
    )
)

fig4a.update_layout(
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

py.plot(fig4a, filename = 'Top 15 Spending', auto_open=True)

'https://plotly.com/~23091998/17/'

In [15]:
# create trace1 
trace1 = go.Bar(# interactive bar object
                x = df.index,
                y = df.Transfer,
                name = " ",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df.Transfer,
                hovertemplate =
                "Club Name: %{x}<br>" +
                "Total Transfer Amount: %{y:,.0f} Euros <br>")

data1 = [trace1]
layout1 = go.Layout(barmode = "group")
fig1 = go.Figure(data = data1, layout = layout1)
fig1.update_layout(
    title=go.layout.Title(
        text="Top 15 Clubs by Transfer Fee Spending (2005-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Club Name",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
        showline = True,
        linecolor = 'black',
        showticklabels = True,
        tickfont = dict(
            family = 'Arial',
            size = 14,
            color = 'black'
      )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Transfer Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
         showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
      )
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

plot(fig1, filename = 'Top 15 Clubs by Transfer Fee Spending.html') #use plot when in lab and iplot(fig) when in tree

'Top 15 Clubs by Transfer Fee Spending.html'

In [8]:
# Top 20 clubs by surplus(over spending)
df1=Test_EDA.groupby("Club Joined").agg({"Surplus/Deficit":"sum"})
df1 = df1.sort_values(by='Surplus/Deficit', ascending = False)
df1 = df1[:20]
df1

,Surplus/Deficit
Club Joined,
Man City,479365000.0
Man Utd,408780000.0
Liverpool,332073000.0
Chelsea,295255000.0
FC Barcelona,249770000.0
Paris SG,244100000.0
Arsenal,201060000.0
Spurs,160620000.0
Aston Villa,155780000.0


In [9]:
merged_df_y = pd.merge(df1.reset_index(),Test_EDA.reset_index(),
                    how='inner',
                    on = 'Club Joined')
merged_df_y_2 = merged_df_x.groupby(['Club Joined', 'Year']).agg({'Surplus/Deficit':'sum'})
merged_df_y_2

Surplus/Deficit
Club Joined Year                 
AC Milan    2005      -19100000.0
            2006        6000000.0
            2007      -13100000.0
            2008      -19600000.0
            2009       -8400000.0
...                           ...
Spurs       2015       28730000.0
            2016       41400000.0
            2017       47100000.0
            2018         400000.0
            2019       -9000000.0

[221 rows x 1 columns]

In [10]:
fig5a = px.line(merged_df_y_2.reset_index(), 
                x="Year", 
                y="Surplus/Deficit", 
                color='Club Joined', 
                hover_name="Club Joined", 
                title = 'Trends for Top 20 Surplus Clubs')

plot(fig5a ,filename = 'TrendsTop20Surplus.html')

'TrendsTop20Surplus.html'

In [11]:
# create trace2
trace2 = go.Bar(# interactive bar object
                x = df1.index,
                y = df1['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df1['Surplus/Deficit'],
                hovertemplate =
                "Club Name: %{x}<br>" +
                "Total Over-Spent Amount: %{y:,.0f} Euros")

data2 = [trace2]
layout2 = go.Layout(barmode = "group")
fig2 = go.Figure(data = data2, layout = layout2)
fig2.update_layout(
    title=go.layout.Title(
        text="Top 10 Clubs by Transfer over-spending (2005-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Club Name",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
         showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)

plot(fig2, filename = 'Top 10 Clubs by Transfer Over-Spending.html') #use plot when in lab and iplot(fig) when in tree

'Top 10 Clubs by Transfer Over-Spending.html'

In [12]:
# Top 10 clubs by deficit (under spending)
df2=Test_EDA.groupby("Club Joined").agg({"Surplus/Deficit":"sum"})
df2 = df2.sort_values(by='Surplus/Deficit', ascending = True)
df2 = df2[:20]
df2

,Surplus/Deficit
Club Joined,
Besiktas,-129775000.0
Galatasaray,-124379000.0
Trabzonspor,-116623000.0
Fenerbahce,-99833000.0
Flamengo,-77610000.0
Antalyaspor,-76733000.0
Konyaspor,-73990000.0
Ankaragücü,-71600000.0
QPR,-68070000.0


In [13]:
# create trace3
trace3 = go.Bar(# interactive bar object
                x = df2.index,
                y = df2['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df2['Surplus/Deficit'],
                hovertemplate =
                "Club Name: %{x}<br>" +
                "Total Over-Spent Amount: %{y:,.0f} Euros")

data3 = [trace3]
layout3 = go.Layout(barmode = "group")
fig3 = go.Figure(data = data3, layout = layout3)
fig3.update_layout(
    title=go.layout.Title(
        text="Top 10 Clubs by Transfer under-spending (2005-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Club Name",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)

plot(fig3, filename = 'Top 10 Clubs by Transfer Under-Spending.html') #use plot when in lab and iplot(fig) when in tree

'Top 10 Clubs by Transfer Under-Spending.html'

In [14]:
# Top 10 clubs by surplus received 

df3=Test_EDA.groupby("Club Left").agg({"Surplus/Deficit":"sum"})
df3 = df3.sort_values(by='Surplus/Deficit', ascending = False)
df3 = df3[:10]
df3

,Surplus/Deficit
Club Left,
Benfica,281555000.0
Monaco,274580000.0
Southampton,185955000.0
Athletic,111600000.0
Shakhtar D.,100000000.0
Leicester,97455000.0
FC Porto,79690000.0
Swansea,78435000.0
Atalanta,77275500.0


In [15]:
# create trace4
trace4 = go.Bar(# interactive bar object
                x = df3.index,
                y = df3['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df3['Surplus/Deficit'],
                hovertemplate =
                "Club Name: %{x}<br>" +
                "Total Over-Spent Amount: %{y:,.0f} Euros")

data4 = [trace4]
layout4 = go.Layout(barmode = "group")
fig4 = go.Figure(data = data4, layout = layout4)
fig4.update_layout(
    title=go.layout.Title(
        text="Top 10 Clubs by Transfer Surplus received (2005-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Club Name",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)

plot(fig4, filename = 'Top 10 Clubs by Transfer Surplus.html') #use plot when in lab and iplot(fig) when in tree

'Top 10 Clubs by Transfer Surplus.html'

In [16]:
# Top positions in terms of surplus
df4=Test_EDA.groupby("Position").agg({"Surplus/Deficit":"sum"})
df4 = df4.sort_values(by='Surplus/Deficit', ascending = False)

df4

,Surplus/Deficit
Position,
Right Winger,5.074750e+07
Left Winger,4.519300e+07
Sweeper,-7.950000e+06
Defender,-3.209300e+07
Forward,-5.877000e+07
Midfielder,-9.151200e+07
Second Striker,-2.052100e+08
Left Midfield,-2.827550e+08
Right Midfield,-2.957570e+08


In [17]:
# create trace5
trace5 = go.Bar(# interactive bar object
                x = df4.index,
                y = df4['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df4['Surplus/Deficit'],
                hovertemplate =
                "Player Position: %{x}<br>" +
                "Surplus/Deficit Spent: %{y:,.0f} Euros")

data5 = [trace5]
layout5 = go.Layout(barmode = "group")
fig5 = go.Figure(data = data5, layout = layout5)
fig5.update_layout(
    title=go.layout.Title(
        text="Transfer Surplus/Deficit by Position (2010-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Position",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)
plot(fig5, filename = 'Transfer Surplus or Deficit by Position.html') #use plot when in lab and iplot(fig) when in tree

'Transfer Surplus or Deficit by Position.html'

In [18]:
# Year-wise average Transfer data trend
Trend=Test_EDA.groupby("Year").agg({"Transfer":np.mean})
Trend

,Transfer
Year,
2005,9.942693e+05
2006,9.801514e+05
2007,1.253471e+06
2008,1.211738e+06
2009,1.073434e+06
2010,6.821355e+05
2011,7.934357e+05
2012,5.943953e+05
2013,6.591834e+05


In [19]:
# Year-wise trend for player valuation
Trend1=Test_EDA.groupby("Year").agg({"Market Valuation":np.mean})
Trend1

,Market Valuation
Year,
2005,1.544702e+06
2006,1.504357e+06
2007,1.454310e+06
2008,1.433811e+06
2009,1.279467e+06
2010,9.942248e+05
2011,1.092635e+06
2012,9.047528e+05
2013,8.600490e+05


In [20]:
# Year-wise trend for surplus/deficit
Trend2=Test_EDA.groupby("Year").agg({"Surplus/Deficit":np.mean})
Trend2

,Surplus/Deficit
Year,
2005,-550432.682425
2006,-525405.528053
2007,-203123.076923
2008,-223936.187683
2009,-206032.424812
2010,-312089.302326
2011,-299198.823719
2012,-310560.401292
2013,-200865.521213


In [21]:
Test_EDA.groupby("Year").agg({"Transfer": ['mean', 'min', 'max', 'var']})

Transfer                                
              mean  min          max           var
Year                                              
2005  9.942693e+05  0.0   38000000.0  1.162948e+13
2006  9.801514e+05  0.0   43880000.0  9.811232e+12
2007  1.253471e+06  0.0   38000000.0  1.411339e+13
2008  1.211738e+06  0.0   43000000.0  1.274943e+13
2009  1.073434e+06  0.0   94000000.0  1.970572e+13
2010  6.821355e+05  0.0   40000000.0  6.836758e+12
2011  7.934357e+05  0.0   58500000.0  1.079620e+13
2012  5.943953e+05  0.0   42000000.0  6.363270e+12
2013  6.591834e+05  0.0  101000000.0  1.238263e+13
2014  5.709954e+05  0.0   81720000.0  1.047529e+13
2015  6.507384e+05  0.0   76000000.0  1.156869e+13
2016  7.485612e+05  0.0  105000000.0  1.442989e+13
2017  8.664987e+05  0.0  222000000.0  2.462475e+13
2018  8.514616e+05  0.0  145000000.0  2.600989e+13
2019  9.507049e+05  0.0  126000000.0  2.625949e+13

In [22]:
Trend_Combined = pd.concat([Trend, Trend1, Trend2], axis=1, join='inner')
Trend_Combined

,Transfer,Market Valuation,Surplus/Deficit
Year,,,
2005,9.942693e+05,1.544702e+06,-550432.682425
2006,9.801514e+05,1.504357e+06,-525405.528053
2007,1.253471e+06,1.454310e+06,-203123.076923
2008,1.211738e+06,1.433811e+06,-223936.187683
2009,1.073434e+06,1.279467e+06,-206032.424812
2010,6.821355e+05,9.942248e+05,-312089.302326
2011,7.934357e+05,1.092635e+06,-299198.823719
2012,5.943953e+05,9.047528e+05,-310560.401292
2013,6.591834e+05,8.600490e+05,-200865.521213


In [23]:
# create trace5
trace6 = go.Bar(# interactive bar object
                x = Trend_Combined.index,
                y = Trend_Combined['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = Trend_Combined['Surplus/Deficit'],
                hovertemplate =
                "Year: %{x}<br>" +
                "Surplus/Deficit: %{y:,.0f} Euros")
trace7 = go.Line(# interactive bar object
                x = Trend_Combined.index,
                y = Trend_Combined.Transfer,
                name = "",
                marker = dict(color = 'rgb(0,153,76)'),
                text = Trend_Combined['Transfer'],
                hovertemplate =
                "Year: %{x}<br>" +
                "Transfer Fee: %{y:,.0f} Euros")
trace8 = go.Line(# interactive bar object
                x = Trend_Combined.index,
                y = Trend_Combined['Market Valuation'],
                name = "",
                marker = dict(color = 'rgb(153,0,0)'),
                text = Trend_Combined['Market Valuation'],
                hovertemplate =
                "Year: %{x}<br>" +
                "Market Valuation: %{y:,.0f} Euros")

data6 = [trace6, trace7, trace8]
layout6 = go.Layout(barmode = "group")
fig6 = go.Figure(data = data6, layout = layout6)
fig6.update_layout(
    title=go.layout.Title(
        text="Time_Series",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)
plot(fig6, filename = 'Time_Series.html') #use plot when in lab and iplot(fig) when in tree

C:\ProgramData\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




'Time_Series.html'

In [24]:
df5=Test_EDA.groupby("Age").agg({"Transfer":np.mean})
df5 = df5.sort_values(by='Transfer', ascending = False)

df5

,Transfer
Age,
18.0,1.362284e+06
23.0,1.343344e+06
16.0,1.336771e+06
24.0,1.328558e+06
22.0,1.292614e+06
21.0,1.280965e+06
19.0,1.271838e+06
25.0,1.212498e+06
20.0,1.139729e+06


In [25]:
# create trace1 
trace9 = go.Bar(# interactive bar object
                x = df5.index,
                y = df5.Transfer,
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = df5.Transfer,
                hovertemplate =
                "Age: %{x}<br>" +
                "Transfer Fee: %{y:,.0f} Euros")

data7 = [trace9]
layout7 = go.Layout(barmode = "group")
fig7 = go.Figure(data = data7, layout = layout7)
fig7.update_layout(
    title=go.layout.Title(
        text="Age-wise Transfer Fee (2005-2019)",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Age",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Average Transfer Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    )
)
plot(fig7, filename = 'Age-wise Average Transfer Fee.html') #use plot when in lab and iplot(fig) when in tree

'Age-wise Average Transfer Fee.html'

In [26]:
# Age-wise no. of transfers and average surplus/deficit

df6=Test_EDA.groupby("Age").agg({"Transfer":"count", "Surplus/Deficit":"mean"})
df6

,Transfer,Surplus/Deficit
Age,,
14.0,1,-25000.000000
15.0,5,-230000.000000
16.0,48,813854.166667
17.0,193,500290.155440
18.0,689,669534.107402
19.0,1572,491468.702290
20.0,2445,390209.734151
21.0,3159,285056.853435
22.0,3673,217211.543697


In [27]:
Age_Analysis = pd.merge(df6, df5, 
                     how = 'outer',
                     on = 'Age',
                    suffixes = ('_Count','_mean'))
Age_Analysis

,Transfer_Count,Surplus/Deficit,Transfer_mean
Age,,,
14.0,1,-25000.000000,0.000000e+00
15.0,5,-230000.000000,0.000000e+00
16.0,48,813854.166667,1.336771e+06
17.0,193,500290.155440,7.854974e+05
18.0,689,669534.107402,1.362284e+06
19.0,1572,491468.702290,1.271838e+06
20.0,2445,390209.734151,1.139729e+06
21.0,3159,285056.853435,1.280965e+06
22.0,3673,217211.543697,1.292614e+06


In [28]:
# create trace5
trace10 = go.Line(# interactive bar object
                x = Age_Analysis.index,
                y = Age_Analysis.Transfer_Count,
                name = "",
                marker = dict(color = 'rgb(153,0,0)'),
                text = Age_Analysis.Transfer_Count,
                hovertemplate =
                "Age: %{x}<br>" +
                "Number of Transfers: %{y:,.0f}",
                yaxis='y2')
trace11 = go.Bar(# interactive bar object
                x = Age_Analysis.index,
                y = Age_Analysis['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,153,76)'),
                text = Age_Analysis['Surplus/Deficit'],
                hovertemplate =
                "Age: %{x}<br>" +
                "Surplus/Deficit: %{y:,.0f}",
                yaxis='y1')
trace12 = go.Bar(# interactive bar object
                x = Age_Analysis.index,
                y = Age_Analysis.Transfer_mean,
                name = "",
                marker = dict(color = 'rgb(0,25,51)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = Age_Analysis.Transfer_mean,
                hovertemplate =
                "Age: %{x}<br>" +
                "Average Transfer Fee: %{y:,.0f}",
                yaxis='y1')

data10 = [trace10, trace11, trace12]
layout10 = go.Layout(barmode = "group")
fig10 = go.Figure(data = data10, layout = layout10)
fig10.update_layout(
    title=go.layout.Title(
        text="Age Analysis",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Age",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
      yaxis2=dict(title='Number of Transfers',
           overlaying='y',
           side='right'),
           font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
showticklabels = True,
tickfont = dict(
family = 'Arial',
size = 14,
color = 'black'
        )

plot(fig10, filename = 'Age_Analysis.html') #use plot when in lab and iplot(fig) when in tree

'Age_Analysis.html'

In [29]:
# top 10 countries with most no. of transfers 
df7= Test_EDA.groupby("Nationality").agg({"Transfer":'count'})
df7 = df7.sort_values(by='Transfer', ascending = False)
df7

,Transfer
Nationality,
Turkey,5496
Brazil,5192
Italy,5149
England,3301
Argentina,1819
...,...
Tahiti,1
Cambodia,1
Somalia,1


In [30]:
# top 10 countries based on average transfer amount
df8= Test_EDA.groupby("Nationality").agg({"Transfer":'mean'})
df8 = df8.sort_values(by='Transfer', ascending = False)
df8

,Transfer
Nationality,
Central African Republic,3.453600e+06
Mexico,3.422602e+06
Portugal,3.163438e+06
Colombia,3.008681e+06
Belgium,2.990378e+06
...,...
Réunion,0.000000e+00
Saint-Martin,0.000000e+00
San Marino,0.000000e+00


In [31]:
# top 10 countries based on average surplus
df9=Test_EDA.groupby("Nationality").agg({"Surplus/Deficit":'mean'})
df9 = df9.sort_values(by='Surplus/Deficit', ascending = False)
df9

,Surplus/Deficit
Nationality,
United Arab Emirates,8.888889e+05
China,8.763265e+05
Jersey,6.800000e+05
Central African Republic,6.046000e+05
Colombia,5.417254e+05
...,...
French Guiana,-7.566667e+05
Sao Tome and Principe,-7.683333e+05
Saint-Martin,-8.857143e+05


In [32]:
Nation_Analysis_Intermediate = pd.merge(df7, df8, 
                     how = 'inner',
                     on = 'Nationality',
                    suffixes = ('_Count','_Mean'))
Nation_Analysis_Intermediate

,Transfer_Count,Transfer_Mean
Nationality,,
Turkey,5496,1.039898e+05
Brazil,5192,1.140306e+06
Italy,5149,5.592858e+05
England,3301,1.016174e+06
Argentina,1819,1.712362e+06
...,...,...
Tahiti,1,1.200000e+06
Cambodia,1,0.000000e+00
Somalia,1,0.000000e+00


In [33]:
Nation_Analysis = pd.merge(Nation_Analysis_Intermediate, df9, 
                     how = 'inner',
                     on = 'Nationality',
                    suffixes = ('_Count','_Mean'))
Nation_Analysis_Top25 = Nation_Analysis[:25]
Nation_Analysis_Top25

,Transfer_Count,Transfer_Mean,Surplus/Deficit
Nationality,,,
Turkey,5496,1.039898e+05,-284207.969432
Brazil,5192,1.140306e+06,-143586.479199
Italy,5149,5.592858e+05,-80842.406292
England,3301,1.016174e+06,95732.505301
Argentina,1819,1.712362e+06,-160037.658054
France,1569,2.764709e+06,-81655.194391
Germany,1469,1.142332e+06,-209370.319946
Spain,1420,2.457454e+06,-154728.802817
Greece,1407,1.638060e+05,-244402.985075


In [34]:
# create trace5
trace13 = go.Line(# interactive bar object
                x = Nation_Analysis_Top25.index,
                y = Nation_Analysis_Top25.Transfer_Count,
                name = "",
                marker = dict(color = 'rgb(153,0,0)'),
                text = Nation_Analysis_Top25.Transfer_Count,
                hovertemplate =
                "Home Country: %{x}<br>" +
                "Number of Transfers: %{y:,.0f}",
                yaxis='y2')
trace14 = go.Bar(# interactive bar object
                x = Nation_Analysis_Top25.index,
                y = Nation_Analysis_Top25['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(0,153,76)'),
                text = Nation_Analysis_Top25['Surplus/Deficit'],
                hovertemplate =
                "Home Country: %{x}<br>" +
                "Surplus/Deficit: %{y:,.0f}",
                yaxis='y1')
trace15 = go.Bar(# interactive bar object
                x = Nation_Analysis_Top25.index,
                y = Nation_Analysis_Top25.Transfer_Mean,
                name = "Average Transfer Value",
                marker = dict(color = 'rgb(0,25,51)'),
                text = Nation_Analysis_Top25.Transfer_Mean,
                hovertemplate =
                "Home Country: %{x}<br>" +
                "Average Transfer Fee: %{y:,.0f}",
                yaxis='y1')

data11 = [trace13, trace14, trace15]
layout11 = go.Layout(barmode = "group")
fig11 = go.Figure(data = data11, layout = layout11)
fig11.update_layout(
    title=go.layout.Title(
        text="Nationality based Analysis",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Nationality",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis2=dict(title='Number of Transfers',
           overlaying='y',
           side='right'),
           font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
showticklabels = True,
tickfont = dict(
family = 'Arial',
size = 14,
color = 'black'
        )

plot(fig11, filename = 'Nation_Analysis_Top25.html') #use plot when in lab and iplot(fig) when in tree

'Nation_Analysis_Top25.html'

In [35]:
China=Test_EDA[Test_EDA.Nationality=='China']

In [36]:
China_Age1=China.groupby("Age").agg({"Transfer":"count", "Surplus/Deficit":"mean"})
China_Age2=China.groupby("Age").agg({"Transfer":np.mean})
China_Age = pd.merge(China_Age1, China_Age2, 
                     how = 'outer',
                     on = 'Age',
                    suffixes = ('_Count','_mean'))
China_Age

,Transfer_Count,Surplus/Deficit,Transfer_mean
Age,,,
17.0,1,1.415000e+06,1.440000e+06
18.0,4,-2.500000e+04,0.000000e+00
19.0,9,3.888889e+04,6.666667e+04
20.0,32,5.101250e+05,5.851250e+05
21.0,33,6.615455e+05,7.554848e+05
22.0,40,4.545250e+05,5.464000e+05
23.0,49,1.202184e+06,1.372592e+06
24.0,58,8.059310e+05,9.481724e+05
25.0,45,1.036378e+06,1.210267e+06


In [37]:
# create trace5
trace21 = go.Line(# interactive bar object
                x = China_Age.index,
                y = China_Age.Transfer_Count,
                name = "",
                marker = dict(color = 'rgb(0,25,51)'),
                text = China_Age.Transfer_Count,
                hovertemplate =
                "Age: %{x}<br>" +
                "Number of Transfers: %{y:,.0f}",
                yaxis='y2')
trace22 = go.Bar(# interactive bar object
                x = China_Age.index,
                y = China_Age['Surplus/Deficit'],
                name = "",
                marker = dict(color = 'rgb(173,0,0)'),
                text = China_Age['Surplus/Deficit'],
                hovertemplate =
                "Age: %{x}<br>" +
                "Surplus/Deficit: %{y:,.0f}",
                yaxis='y1')
trace23 = go.Bar(# interactive bar object
                x = China_Age.index,
                y = China_Age.Transfer_mean,
                name = "",
                marker = dict(color = 'teal'),
                text = China_Age.Transfer_mean,
                hovertemplate =
                "Age: %{x}<br>" +
                "Average Transfer Value: %{y:,.0f}",
                yaxis='y1')

data14 = [trace21, trace22, trace23]
layout14 = go.Layout(barmode = "group")
fig14 = go.Figure(data = data14, layout = layout14)
fig14.update_layout(
    title=go.layout.Title(
        text="China Age Analysis",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Age",
            font=dict(
                family="Arial, monospace",
                size=18,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Total Amount (in EUR)",
            font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
    showticklabels = True,
      tickfont = dict(
      family = 'Arial',
      size = 14,
      color = 'black'
        )
    ),
      yaxis2=dict(title='Number of Transfers',
           overlaying='y',
           side='right'),
           font=dict(
                family="Arial, monospace",
                size=14,
                color="black"
            )
        ),
showticklabels = True,
tickfont = dict(
family = 'Arial',
size = 14,
color = 'black'
        )

plot(fig14, filename = 'China_Age.html') #use plot when in lab and iplot(fig) when in tree

'China_Age.html'